### Importación de librerías

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dateutil import parser
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
mmscaler = MinMaxScaler()
sscaler = StandardScaler()
rscaler = RobustScaler()
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture
from sklearn.cluster import Birch
import gc
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

### Lectura del archivo excel con la información de los clientes

In [3]:
df_tabla_final = pd.read_excel("../archivos_entrada_script/tabla_final.xlsx", sheet_name="Hoja2", dtype={"CODIGOCLIE":str})
df_tabla_final["CODIGOCLIE"] = df_tabla_final["CODIGOCLIE"].astype("string")
df_tabla_final = df_tabla_final.iloc[:,2:6]
df_tabla_final

,Nombre,CIIU,CODIGOCLIE,Mult. de potencias
0,LA INDUSTRIA HARINERA S.A,C1061.11,1401867926,1100.0
1,CONFITECA,C1073.21,1490000744,1100.0
2,SUCESORES DE JACOBO PAREDES M. S.A.,C1074.01,1490000688,1100.0
3,"AC BEBIDAS, S. DE R.L. DE C.V.",C1104.01,1490001175,400.0
4,S.J. JERSEY ECUATORIANO C.A.,C1311.02,1490000515,800.0
...,...,...,...,...
209,CIRION TECHNOLOGIES ECUADOR S.A.,J6190.02,1410020880,1140.0
210,ECUATORIANA DE ARTEFACTOS S.A. ECASA,C2750.01,1490000693,1.0
211,INTEXDECOR S.A.,G4641.11,1490001644,1000.0
212,INTELA INDUSTRIA TEXTIL LATINOAMERICANA CIA. L...,C1312.01,1410062892,1100.0


In [9]:
clientes_unicos = df_tabla_final["CODIGOCLIE"].unique()
print(clientes_unicos)

<StringArray>
['1401867926', '1490000744', '1490000688', '1490001175', '1490000515',
 '1401337167', '1490002225', '1401582655', '1490001247', '1401282968',
 ...
 '1401229975', '1401954998', '1490000719', '1401940609', '1410026628',
 '1410020880', '1490000693', '1490001644', '1410062892', '1490002155']
Length: 214, dtype: string


In [10]:
def intentar_abrir_archivo_datos(path_archivo):

    df_archivo_telem_tab = pd.DataFrame([1,2,3], columns=["prueba"])
    df_archivo_telem_pc = pd.DataFrame([1,2,3], columns=["prueba"])
    df_archivo_telem_c = pd.DataFrame([1,2,3], columns=["prueba"])

    # Intentar leer el archivo con separador 'tab'
    try:
        #print("Leyendo con tab")
        df_archivo_telem_tab = pd.read_csv(path_archivo,
                                            sep="\t",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            ) 
    except:
        pass

    # Intentar leer el archivo con separador 'punto y coma'
    try:
        #print("Leyendo con ;")
        df_archivo_telem_pc = pd.read_csv(path_archivo,
                                            sep=";",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            #encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            )

    except:
        pass

    # Intentar leer el archivo con separador 'coma'
    try:
        #print("Leyendo con ,")
        df_archivo_telem_c = pd.read_csv(path_archivo,
                                            sep=",",
                                            #decimal=",",
                                            skiprows=11,
                                            #na_values="N/D",
                                            #encoding="utf-16",
                                            #on_bad_lines="skip",
                                            encoding_errors="ignore" 
                                            )

    except:
        pass



    if len(df_archivo_telem_tab.columns) > 1:
        df_archivo_telem = df_archivo_telem_tab.copy()
    elif len(df_archivo_telem_pc.columns) > 1:
        df_archivo_telem = df_archivo_telem_pc.copy()
    else:
        df_archivo_telem = df_archivo_telem_c.copy()

    return df_archivo_telem

### 1. Extracción de los archivos de cada grupo de clientes

#### 1.1 Extracción de los archivos con las mediciones mensuales del primer grupo de clientes

Estos clientes están clasificados por CUEN, y el formato de sus archivos varían, el procedimiento a seguir será:
1. Iterar sobre cada código de cliente
2. Buscar sus archivos de datos mensuales
3. Unificar todos en un solo archivo
4. El identificador a usar es el código de cliente

In [13]:
mediciones_clientes_g1 = r"../data/mediciones_por_mes_g1"
archivos_mediciones_g1 = list(os.scandir(mediciones_clientes_g1))
columnas_extraer_g1 = ["Fecha","Demanda activa DEL","Demanda reactiva DEL"]

In [14]:
clientes_unicos_g1 = set()

for medicion in archivos_mediciones_g1:
    cliente = medicion.name.split('-')[1]
    clientes_unicos_g1.add(cliente)

print(f"Clientes único encontrados en la carpeta de mediciones del grupo uno: {len(clientes_unicos_g1)}")

Clientes único encontrados en la carpeta de mediciones del grupo uno: 314


In [15]:
# Diccionario para almacenar los clientes con sus respectivos datos
dict_dfs_clientes_g1 = {}

# Iterar sobre cada cliente
for cliente in tqdm(clientes_unicos_g1, desc="Procesando clientes del grupo 01"):
    datos_cliente = []
    # Iterar sobre cada archivo de medicion del grupo 01
    for medicion in archivos_mediciones_g1:
        if cliente == medicion.name.split("-")[1]:
            df_cliente = intentar_abrir_archivo_datos(f"{mediciones_clientes_g1}/{medicion.name}")
            datos_cliente.extend(df_cliente[columnas_extraer_g1].values)

    # Convertir a DataFrame los datos concatenados
    df_datos_anual_cliente = pd.DataFrame(datos_cliente, columns=columnas_extraer_g1)
    
    # Almacenar en el diccionario (Clave->Cliente   Valor->DataFrame)
    dict_dfs_clientes_g1[cliente]=df_datos_anual_cliente
    
    # Eliminar dataframe concatenado para liberar memoria
    del df_datos_anual_cliente
    #df_datos_anual_cliente.to_csv(f"mediciones_por_anio/g1_perfil_carga_anual-{cliente}-2023.csv", index=False)

Procesando clientes del grupo 01: 100%|██████████| 314/314 [01:47<00:00,  2.91it/s]


#### 1.2 Extracción de los archivos con las mediciones mensuales del segundo grupo de clientes

De estos clientes tenemos carpetas con sus mediciones por mes, no existe tabla de excel inicial, se procederá a realizar lo siguiente:
1. Iterar sobre cada carpeta (cliente)
2. Obtener los datos de sus 12 meses
3. Unificar en un único archivo anual
4. Se usará el nombre del cliente como identificador

In [17]:
mediciones_clientes_g2 = "../data/mediciones_por_mes_g2"
archivos_mediciones_g2 = list(os.scandir(mediciones_clientes_g2))
columnas_extraer_g2 = ["Fecha", "AS (kWh)"]

In [18]:
print(f"Clientes único encontrados en la carpeta de mediciones del grupo uno: {len(archivos_mediciones_g2)}")

Clientes único encontrados en la carpeta de mediciones del grupo uno: 75


In [19]:
# Diccionario para almacenar los clientes con sus datos
dict_dfs_clientes_g2 = {}

# Iterar sobre cada cliente
for archivos_cliente in tqdm(archivos_mediciones_g2, desc="Procesando clientes del grupo 02"):
    nombre_cli = archivos_cliente.name.strip()
    df_concat = pd.DataFrame()

    # Obtener los archivos de las mediciones mensuales del cliente
    mediciones_mensuales_cliente = os.scandir(rf"{mediciones_clientes_g2}/{nombre_cli}")

    for medicion in mediciones_mensuales_cliente:
        medicion_mensual = pd.read_csv(rf"{mediciones_clientes_g2}/{nombre_cli}/{medicion.name}", sep=";", skiprows=2, encoding='ISO-8859-1')
        medicion_mensual = medicion_mensual[columnas_extraer_g2]
        df_concat = pd.concat([df_concat, medicion_mensual])

    # Almacenar en el diccionario (Clave->Cliente   Valor->DataFrame)
    dict_dfs_clientes_g2[nombre_cli] = df_concat
    
    # Eliminar dataframe concatenado para liberar memoria
    del df_concat
    #df_concat.to_csv(rf"mediciones_por_anio/g2_perfil_carga_anual-{nombre_cli}-2023.csv", index=False)

Procesando clientes del grupo 02: 100%|██████████| 75/75 [00:19<00:00,  3.82it/s]


### 2. Preprocesamiento de los datos

Ahora tenemos todos los datos unificados anualmente por cada cliente, tenemos que limpiar y preprocesar, realizar las siguientes tareas:
1. Calcular la potencia aparente (resultado de aplicar teorema pitágoras sobre potencia activa y reactiva)
2. Separar la columna 'fecha' en dos columnas 'fecha' y 'hora', fecha va a tener formato 'año/mes/dia' y hora el formato 'hh:mm'
3. Excluir aquellos registros que correspondan a fechas de sábado, domingo o días de feriado nacional
4. Normalizar los datos para que todos estén en la misma escala

In [20]:
feriados_nacionales = ["2/7/2023", "20/2/2023", "21/2/2023", "7/4/2023", "1/5/2023", \
                       "26/5/2023", "11/8/2023", "9/10/2023", "2/10/2023", "3/10/2023", "25/12/2023"]
feriados_nacionales = pd.to_datetime(feriados_nacionales, format='%d/%m/%Y')

dict_meses = {"01": "Enero",
              "02": "Febrero",
              "03": "Marzo",
              "04": "Abril",
              "05": "Mayo",
              "06": "Junio",
              "07": "Julio",
              "08": "Agosto",
              "09": "Septiembre",
              "10": "Octubre",
              "11": "Noviembre",
              "12": "Diciembre"}

print(list(dict_meses.keys()))
print(feriados_nacionales)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
DatetimeIndex(['2023-07-02', '2023-02-20', '2023-02-21', '2023-04-07',
               '2023-05-01', '2023-05-26', '2023-08-11', '2023-10-09',
               '2023-10-02', '2023-10-03', '2023-12-25'],
              dtype='datetime64[ns]', freq=None)


In [21]:
def fecha_formato_unico(fecha_str):

    # Poner separador único el '/' y año únicamente 2023
    fecha_str = fecha_str.replace('-','/').replace('2024','2023')

    if len(fecha_str.split('/')[0]) == 4: # Cuando el formato es año/mes/día
        return fecha_str
    elif len(fecha_str.split('/')[0]) != 4: # Cuando el formato es día/mes/año
        seps = fecha_str.split('/')
        return f"{seps[-1]}/{seps[1]}/{seps[0]}"
    

def recortar_valores_atipicos(df, columna_valor):
    # Calcular los cuartiles y el IQR
    Q1 = df[columna_valor].quantile(0.25)
    Q3 = df[columna_valor].quantile(0.75)
    IQR = Q3 - Q1

    # Definir los límites superior e inferior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Aplicar el recorte
    df[columna_valor] = df[columna_valor].clip(lower=limite_inferior, upper=limite_superior)

2.1 - Transformación columna fecha archivos anuales grupo 01

In [22]:
dict_dfs_procesados_g1 = {}

for cliente, df_archivo_g1 in dict_dfs_clientes_g1.items():

    # Transformar columna fecha a cadena
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].astype("string")

    # Eliminar los valores nulos en la columna 'Fecha'
    df_archivo_g1 = df_archivo_g1.dropna(subset="Fecha")

    # Separar para obtener columna Hora
    df_archivo_g1["Hora"] = df_archivo_g1["Fecha"].apply(lambda x: x.split()[1].strip())

    # Separar para obtener columna Fecha
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].apply(lambda x: x.split()[0].strip())

    # Transformar la columna Fecha a un formato único
    df_archivo_g1["Fecha"] = df_archivo_g1["Fecha"].apply(fecha_formato_unico)

    # Debido a que a veces se ponen datos del 2024 para reemplazar los faltantes del 2023
    # debemos descartar la fecha 29 de febrero, pues en 2023 no existe
    df_archivo_g1 = df_archivo_g1[df_archivo_g1["Fecha"] != "2023/02/29"]
    
    # Eliminar duplicados y conservar el original del 2023
    df_archivo_g1["Fecha-Hora"] = df_archivo_g1["Fecha"] + " " + df_archivo_g1["Hora"]
    df_archivo_g1 = df_archivo_g1.drop_duplicates(subset="Fecha-Hora", keep="first")

    # Transformar columna Fecha a datetime
    df_archivo_g1["Fecha"] = pd.to_datetime(df_archivo_g1["Fecha"], format='%Y/%m/%d')

    # Obtener la potencia aparente
    df_archivo_g1["Potencia_aparente"] = np.sqrt((df_archivo_g1["Demanda activa DEL"]**2) + (df_archivo_g1["Demanda reactiva DEL"]**2))

    # Eliminar días feriados y días de fin de semana
    df_archivo_g1 = df_archivo_g1[~df_archivo_g1['Fecha'].isin(feriados_nacionales) & ~df_archivo_g1['Fecha'].dt.weekday.isin([5, 6])]

    # Interpolar valores nulos usando una función polinomial
    df_archivo_g1["Potencia_aparente"] = df_archivo_g1["Potencia_aparente"].interpolate(method='polynomial', order=3)

    # Conservar solo las columnas de interés
    df_archivo_g1 = df_archivo_g1[["Fecha", "Hora", "Potencia_aparente"]]

    # Escalar las mediciones
    df_archivo_g1["Potencia_aparente_escalada"] = mmscaler.fit_transform(df_archivo_g1[["Potencia_aparente"]])

    # Guardar en un nuevo diccionario los datos procesados
    dict_dfs_procesados_g1[cliente] = df_archivo_g1
    
    # Liberar memoria
    del df_archivo_g1
    
del dict_dfs_clientes_g1

2.2 - Transformación columna fecha archivos anuales grupo 02

In [23]:
dict_dfs_procesados_g2 = {}

for cliente, df_archivo_g2 in dict_dfs_clientes_g2.items():

    # Transformar columna fecha a cadena
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].astype("string")

    # Eliminar registros que contienen el total
    df_archivo_g2 = df_archivo_g2[~df_archivo_g2["Fecha"].str.contains("Total")]

    # Eliminar los valores nulos en la columna 'Fecha'
    df_archivo_g2 = df_archivo_g2.dropna(subset="Fecha")

    # Separar para obtener columna Hora
    df_archivo_g2["Hora"] = df_archivo_g2["Fecha"].apply(lambda x: x.split()[1].strip())

    # Separar para obtener columna Fecha
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].apply(lambda x: x.split()[0].strip())

    # Eliminar duplicados y conservar el original del 2023
    df_archivo_g2["Fecha-Hora"] = df_archivo_g2["Fecha"] + " " + df_archivo_g2["Hora"]
    df_archivo_g2["Fecha-Hora"] = pd.to_datetime(df_archivo_g2["Fecha-Hora"], yearfirst=True)
    df_archivo_g2 = df_archivo_g2.drop_duplicates(subset="Fecha-Hora", keep="first")

    # Restar un timedelta de 15 a todas las fechas (Solo en este caso por que la ultima fecha)
    # se pasa al siguiente mes
    df_archivo_g2["Fecha-Hora"] = df_archivo_g2["Fecha-Hora"] - pd.Timedelta(minutes=15)

    # Separar nuevamente para obtener columna Hora
    df_archivo_g2["Hora"] = df_archivo_g2["Fecha-Hora"].astype("string").apply(lambda x: x.split()[1][:-3].strip())

    # Separar nuevamente para obtener columna Fecha
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha-Hora"].astype("string").apply(lambda x: x.split()[0].strip())

    # Transformar la columna Fecha a un formato único
    df_archivo_g2["Fecha"] = df_archivo_g2["Fecha"].apply(fecha_formato_unico)

    # Debido a que a veces se ponen datos del 2024 para reemplazar los faltantes del 2023
    # debemos descartar la fecha 29 de febrero, pues en 2023 no existe
    df_archivo_g2 = df_archivo_g2[df_archivo_g2["Fecha"] != "2023/02/29"]

    # Transformar columna Fecha a datetime
    df_archivo_g2["Fecha"] = pd.to_datetime(df_archivo_g2["Fecha"], format='%Y/%m/%d')

    # Limpiar la columna 'SE (KVah)'
    df_archivo_g2["AS (kWh)"] = df_archivo_g2["AS (kWh)"].astype("string").str.replace(",", "").replace('"','')
    df_archivo_g2["AS (kWh)"] = df_archivo_g2["AS (kWh)"].astype("float")

    # Obtener la potencia aparente
    df_archivo_g2["Potencia_aparente"] = df_archivo_g2["AS (kWh)"] * 4

    # Eliminar días feriados y días de fin de semana
    df_archivo_g2 = df_archivo_g2[~df_archivo_g2['Fecha'].isin(feriados_nacionales) & ~df_archivo_g2['Fecha'].dt.weekday.isin([5, 6])]

    # Interpolar valores nulos usando una función polinomial
    df_archivo_g2["Potencia_aparente"] = df_archivo_g2["Potencia_aparente"].interpolate(method='polynomial', order=3)

    # Conservar solo las columnas de interés
    df_archivo_g2 = df_archivo_g2[["Fecha", "Hora", "Potencia_aparente"]]

    # Escalar las mediciones
    df_archivo_g2["Potencia_aparente_escalada"] = mmscaler.fit_transform(df_archivo_g2[["Potencia_aparente"]])

    # Guardar en un nuevo diccionario los datos procesados
    dict_dfs_procesados_g2[cliente] = df_archivo_g2
    
    # Liberar memoria
    del df_archivo_g2
    
del dict_dfs_clientes_g2

### 3. Generación de los entregables

In [24]:
def obtener_coords_curva_tipo(df):

    # Agrupar por hora y aplicar mediana
    df_grouped = df.groupby("Hora")["Potencia_aparente_escalada"].apply(np.mean).sort_index(ascending=True).reset_index(drop=False)

    # Retornar el array con los 96 valores de demanda
    return df_grouped

def obtener_coords_dia_demanda_max(df):

    # Obtener el máximo valor de potencia aparente
    max_potencia = df['Potencia_aparente_escalada'].max()

    # Encontrar la fecha correspondiente a la máxima potencia aparente
    fecha_max_potencia = df[df['Potencia_aparente_escalada'] == max_potencia]['Fecha'].iloc[0]

    # Filtrar los registros correspondientes a esa fecha
    df_max_fecha = df[df['Fecha'] == fecha_max_potencia]

    # Ordenar el resultado de manera ascendente por 'Hora'
    df_max_fecha = df_max_fecha.sort_values(by="Hora", ascending=True)

    return fecha_max_potencia, df_max_fecha[["Hora","Potencia_aparente_escalada"]]

In [38]:
def graficar_curva_tipo(df, cod_cli, path):

    # Generar el gráfico de la curva tipo
    _ = plt.figure(figsize=(16, 6))
    _ = plt.plot(df["Hora"], df["Potencia_aparente_escalada"], marker='o', color='b', linestyle='-', label='Potencia Aparente Escalada')
    _ = plt.title(f'Curva tipo cliente {cod_cli}')
    _ = plt.xlabel('Hora')
    _ = plt.ylabel('Potencia Aparente Escalada')
    _ = plt.grid(True)

    # Rotar etiquetas para que no se vea acumulado el eje X
    _ = plt.xticks(df["Hora"].values[::2], rotation=45)

    # Para que no se distorsione la dimenisión del eje Y
    _ = plt.yticks(np.arange(0, 1.1, 0.1))

    # Evitar recortes en las etiquetas
    _ = plt.tight_layout()

    # Guardar la gráfica en un directorio
    _ = plt.savefig(f"{path}/curva_tipo_{cod_cli}.png", format='png')  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    _ = plt.savefig(f"../outputs/curvas_tipo/curva_tipo_{cod_cli}.png", format='png')  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    
    # Cerrar la figura después de guardarla para liberar recursos
    _ = plt.close()  

def graficar_dia_max_demanda(df, cod_cli, path, fecha):

    # Graficar la potencia aparente escalada a lo largo del día
    _ = plt.figure(figsize=(16, 6))
    _ = plt.plot(df["Hora"], df["Potencia_aparente_escalada"], marker='o', color='r', linestyle='-', label=f'Potencia Aparente Escalada')
    _ = plt.title(f'Curva del día de demanda máxima {fecha} para cliente {cod_cli}')
    _ = plt.xlabel('Hora')
    _ = plt.ylabel('Potencia Aparente Escalada')
    _ = plt.grid(True)
    _ = plt.legend()

    # Rotar etiquetas para que no se vea acumulado el eje X
    _ = plt.xticks(df["Hora"].values[::2], rotation=45)

    # Para que no se distorsione la dimenisión del eje Y
    _ = plt.yticks(np.arange(0, 1.1, 0.1))

    # Evitar recortes en las etiquetas
    _ = plt.tight_layout()

    # Guardar la gráfica en un archivo (por ejemplo, como archivo PNG)
    _ = plt.savefig(f"{path}/curva_dia_demanda_max_{cod_cli}.png", format='png')  # Puedes cambiar el formato a 'jpg', 'pdf', etc.
    
    # Cerrar la figura después de guardarla para liberar recursos
    _ = plt.close()

In [26]:
def agrupar_horas(df, columna_hora="Hora", columna_valor="Potencia_aparente_escalada"):

    # Convertir la columna Hora a tipo datetime
    df["Hora"] = pd.to_datetime(df[columna_hora], format="%H:%M")

    # Agrupar por hora y tomar el punto medio (hh:30)
    df["Hora"] = df["Hora"].dt.floor("H") + pd.Timedelta(minutes=30)

    # Recortar a HH:MM
    df["Hora"] = df["Hora"].astype("string").apply(lambda x: x.split()[1][:-3])

    # Agrupar por la nueva columna de hora y calcular el promedio de los valores
    return df.groupby("Hora")[columna_valor].apply(np.mean).reset_index()

In [27]:
def agrupar_30_min(df, columna_hora="Hora", columna_valor="Potencia_aparente_escalada"):

    # Convertir la columna Hora a tipo datetime
    df["Hora"] = pd.to_datetime(df[columna_hora], format="%H:%M")

    # Redondear hacia arriba al final del intervalo de 30 minutos
    df["Hora"] = df["Hora"] + pd.Timedelta(minutes=30)
    df["Hora"] = df["Hora"].dt.floor("H") + (df["Hora"].dt.minute // 30) * pd.Timedelta(minutes=30)

    # Recortar a HH:MM
    df["Hora"] = df["Hora"].astype("string").apply(lambda x: x.split()[1][:-3])

    # Agrupar por la nueva columna de hora y calcular el promedio de los valores
    return df.groupby("Hora")[columna_valor].apply(np.mean).reset_index()

### Proceso de agregación y obtención de las curvas

In [40]:
path_entregables = "../outputs"

#dict_todos_los_clientes = dict_dfs_procesados_g1 | dict_dfs_procesados_g2
#del dict_dfs_procesados_g1
#del dict_dfs_procesados_g2

registros_curvas_todas = []

for cliente, df_medicion_anual in dict_todos_los_clientes.items():

    # Lista para almacenar todos los valores de la curva tipo del cliente
    registros_curva_cliente = []

    # Directorio entregables cliente
    dir_entregables_cli = fr"{path_entregables}/entregables_por_cliente/{cliente}"

    # Crear el directorio para los entregables (Si no existe)
    if not os.path.exists(dir_entregables_cli):
        os.makedirs(os.path.normpath(dir_entregables_cli))

    # Generar el archivo con los datos de la curva tipo
    df_curva_tipo = obtener_coords_curva_tipo(df_medicion_anual)
    df_curva_tipo = agrupar_30_min(df_curva_tipo)
    df_curva_tipo.to_csv(f"{dir_entregables_cli}/datos_curva_tipo_{cliente}.csv", index=False)

    # Guardar los datos en una lista
    registros_curva_cliente.append(cliente)
    for valor in df_curva_tipo["Potencia_aparente_escalada"].values:
        registros_curva_cliente.append(valor)

    # Generar el archivo con los datos de la curva del día que hubo la demanda máxima
    fecha_max_dem, df_curva_dia_dem_max = obtener_coords_dia_demanda_max(df_medicion_anual)
    df_curva_dia_dem_max = df_curva_dia_dem_max.sort_values(by="Hora", ascending=True)
    df_curva_dia_dem_max.to_csv(f"{dir_entregables_cli}/datos_curva_dia_demanda_max.csv", index=False)

    # Generar la gráfica de la curva tipo
    graficar_curva_tipo(df_curva_tipo, cliente, dir_entregables_cli)

    # Generar la gráfica de la curva del día de demanda máxima
    graficar_dia_max_demanda(df_curva_dia_dem_max, cliente, dir_entregables_cli, str(fecha_max_dem).split()[0])

    # Generar un archivo plano con la demanda máximo y mínima
    open(rf'{dir_entregables_cli}/Potencia_max_min.txt', 'w')\
        .write(f'Pot_aparente_max: {df_medicion_anual["Potencia_aparente"].max()}\nPot_aparente_min: {df_medicion_anual["Potencia_aparente"].min()}')

    # Guardar la lista con los registros de un cliente en otra lista
    registros_curvas_todas.append(registros_curva_cliente)

columnas_df_todas_las_curvas = ["Cliente"]
columnas_df_todas_las_curvas.extend(df_curva_tipo["Hora"].values)

### 4. Cargar los datos a MongoDB (únicamente los de curvas tipo)

In [41]:
del dict_todos_los_clientes

In [64]:
df_registros_todas_las_curvas = pd.DataFrame(registros_curvas_todas)
df_registros_todas_las_curvas.columns = columnas_df_todas_las_curvas
df_registros_todas_las_curvas

,Cliente,00:00,00:30,01:00,01:30,02:00,02:30,03:00,03:30,04:00,...,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30
0,1346675,0.098762,0.098195,0.096183,0.095140,0.093804,0.086246,0.086182,0.085850,0.085711,...,0.336088,0.331090,0.326909,0.314862,0.273939,0.223795,0.158809,0.123738,0.113781,0.104419
1,1239960,0.266920,0.245622,0.242817,0.239258,0.239299,0.235997,0.230490,0.224546,0.216955,...,0.374834,0.333382,0.315742,0.308171,0.300832,0.282575,0.268354,0.274110,0.272661,0.269703
2,1548499,0.388375,0.366804,0.368225,0.379075,0.369710,0.366172,0.361147,0.356091,0.355573,...,0.474304,0.471931,0.455084,0.443409,0.434870,0.427804,0.414605,0.403408,0.400265,0.396820
3,90000352,0.417069,0.379594,0.379035,0.380607,0.380429,0.378281,0.376325,0.377653,0.382576,...,0.388154,0.396437,0.400975,0.402848,0.398754,0.404907,0.410273,0.409373,0.411436,0.414023
4,90002399,0.628528,0.626009,0.623813,0.623969,0.622339,0.623259,0.620106,0.616712,0.615479,...,0.598455,0.598969,0.605576,0.618404,0.621603,0.623841,0.626367,0.635738,0.634025,0.632454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384,SIGMAPLAST,0.723962,0.695278,0.696736,0.694418,0.695502,0.693208,0.692603,0.692475,0.693352,...,0.638462,0.627162,0.656135,0.666052,0.672535,0.702778,0.719468,0.720108,0.719691,0.719461
385,SINTOFIL,0.613444,0.528363,0.527601,0.526592,0.525030,0.523573,0.521665,0.518018,0.512443,...,0.613016,0.617259,0.617420,0.616574,0.615983,0.616780,0.614077,0.607881,0.601575,0.608928
386,SOCIEDAD INDUSTRIAL RELI CYRANO,0.522447,0.533413,0.594090,0.619295,0.542844,0.474003,0.453784,0.474405,0.424981,...,0.228917,0.220160,0.220309,0.217405,0.226077,0.253856,0.331315,0.413486,0.496200,0.534456
387,TEXTILES TEXSA,0.675463,0.596102,0.601228,0.594504,0.587015,0.566832,0.579295,0.588318,0.574999,...,0.538375,0.606685,0.606651,0.577992,0.615831,0.665055,0.668014,0.661269,0.662274,0.675761


In [65]:
### Verificar valores nulos
df_registros_todas_las_curvas[df_registros_todas_las_curvas.isna().any(axis=1)]

,Cliente,00:00,00:30,01:00,01:30,02:00,02:30,03:00,03:30,04:00,...,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30
63,90000662,0.181778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
### Verificar duplicados
df_registros_todas_las_curvas["Cliente"] = df_registros_todas_las_curvas["Cliente"].astype("string")
df_registros_todas_las_curvas["Cliente"] = df_registros_todas_las_curvas["Cliente"].apply(lambda x: x[1:] if x.startswith('0') else x)
df_registros_todas_las_curvas[df_registros_todas_las_curvas["Cliente"].duplicated(keep=False)]

,Cliente,00:00,00:30,01:00,01:30,02:00,02:30,03:00,03:30,04:00,...,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30
89,90000664,0.08561,0.082046,0.078968,0.077377,0.076017,0.076011,0.075578,0.075733,0.075818,...,0.129012,0.135853,0.136615,0.134978,0.131076,0.128614,0.124315,0.114976,0.103232,0.094857
239,90000664,0.08561,0.082046,0.078968,0.077377,0.076017,0.076011,0.075578,0.075733,0.075818,...,0.129012,0.135853,0.136615,0.134978,0.131076,0.128614,0.124315,0.114976,0.103232,0.094857


In [67]:
### Excluir filas con valores nulos o filas duplicadas
excluir = ['90000662',
           '090000664']
df_registros_todas_las_curvas = df_registros_todas_las_curvas[~df_registros_todas_las_curvas["Cliente"].isin(excluir)]
df_registros_todas_las_curvas = df_registros_todas_las_curvas.drop_duplicates(subset="Cliente")

In [68]:
df_registros_todas_las_curvas.shape

(387, 49)

In [109]:
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo import MongoClient

def obtener_cliente_db():
    # Cargar las variables del archivo .env
    load_dotenv()

    # Obtener las credenciales y el cluster de MongoDB
    username = os.getenv("DB_USER_T")
    password = os.getenv("DB_PASS_T")
    cluster = os.getenv("DB_CLUSTER_T")

    # Construir la uri con las credenciales
    uri = f"mongodb+srv://{username}:{password}@{cluster.lower()}.gypwd.mongodb.net/?retryWrites=true&w=majority&appName={cluster}"

    # Crear un cliente y conectarlo al servidor
    client = MongoClient(uri, 
                        server_api=ServerApi('1'),
                        connectTimeoutMS=60000,
                        socketTimeoutMS=60000,
                        serverSelectionTimeoutMS=60000,
                        tls=True)

    # Si el cliente existe, retornarlo
    if client is not None:
        return client

In [110]:
# Obtener cliente para conectar a la db
db_cliente = obtener_cliente_db()

# Convertir el dataframe a diccionario
datos_insertar = df_registros_todas_las_curvas.to_dict(orient='records')

# Eliminar cualquier documento existente en la colección
db_cliente.CurvasTipo.CurvasTipo_30m.delete_many({})

# Creación de índice para el campo de la fecha
db_cliente.CurvasTipo.CurvasTipo_30m.create_index([("Cliente", 1)])

# Insertar en la base de datos
db_cliente.CurvasTipo.CurvasTipo_30m.insert_many(datos_insertar)

InsertManyResult([ObjectId('67cf4ca3979e8993ab9e5bc3'), ObjectId('67cf4ca3979e8993ab9e5bc4'), ObjectId('67cf4ca3979e8993ab9e5bc5'), ObjectId('67cf4ca3979e8993ab9e5bc6'), ObjectId('67cf4ca3979e8993ab9e5bc7'), ObjectId('67cf4ca3979e8993ab9e5bc8'), ObjectId('67cf4ca3979e8993ab9e5bc9'), ObjectId('67cf4ca3979e8993ab9e5bca'), ObjectId('67cf4ca3979e8993ab9e5bcb'), ObjectId('67cf4ca3979e8993ab9e5bcc'), ObjectId('67cf4ca3979e8993ab9e5bcd'), ObjectId('67cf4ca3979e8993ab9e5bce'), ObjectId('67cf4ca3979e8993ab9e5bcf'), ObjectId('67cf4ca3979e8993ab9e5bd0'), ObjectId('67cf4ca3979e8993ab9e5bd1'), ObjectId('67cf4ca3979e8993ab9e5bd2'), ObjectId('67cf4ca3979e8993ab9e5bd3'), ObjectId('67cf4ca3979e8993ab9e5bd4'), ObjectId('67cf4ca3979e8993ab9e5bd5'), ObjectId('67cf4ca3979e8993ab9e5bd6'), ObjectId('67cf4ca3979e8993ab9e5bd7'), ObjectId('67cf4ca3979e8993ab9e5bd8'), ObjectId('67cf4ca3979e8993ab9e5bd9'), ObjectId('67cf4ca3979e8993ab9e5bda'), ObjectId('67cf4ca3979e8993ab9e5bdb'), ObjectId('67cf4ca3979e8993ab9e5b